## Prototyping live bus feed 
(based on "get the newest parquet in the data lake" method)

In [6]:
system_id="tfnsw_buses"

In [7]:
import boto3
bucket_name="bus-observatory-staging"
prefix = f"feeds/tfnsw_bus"

In [8]:
# after https://stackoverflow.com/questions/45375999/how-to-download-the-latest-file-of-an-s3-bucket-using-boto3

import boto3

def get_most_recent_s3_object(bucket_name, prefix):
    s3 = boto3.client('s3')
    paginator = s3.get_paginator( "list_objects_v2" )
    page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=prefix)
    latest = None
    for page in page_iterator:
        if "Contents" in page:
            latest2 = max(page['Contents'], key=lambda x: x['LastModified'])
            if latest is None or latest2['LastModified'] > latest['LastModified']:
                latest = latest2
    return latest

latest = get_most_recent_s3_object(bucket_name, prefix)
print(latest)

{'Key': 'feeds/tfnsw_bus/INCOMING_tfnsw_bus_2023-03-30_18_33_57.parquet', 'LastModified': datetime.datetime(2023, 3, 30, 22, 34, tzinfo=tzutc()), 'ETag': '"e7c65707c278335b4a7e5cf71892bd8b"', 'Size': 125069, 'StorageClass': 'STANDARD'}


In [9]:
import boto3
import geopandas as gpd
import io

# Set up an Amazon S3 client object
s3 = boto3.client('s3')

# Retrieve the Parquet file from S3

response = s3.get_object(Bucket=bucket_name, Key=latest['Key'])
parquet_object = response['Body'].read()

# Create an in-memory buffer from the Parquet file
buffer = io.BytesIO(parquet_object)

# Read the data from the in-memory buffer and create a GeoDataFrame
gdf = gpd.read_parquet(buffer)

# Work with the GeoDataFrame as required
print(gdf.head())


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 7: invalid start byte

In [ ]:
# read it into a geodataframe
# https://janakiev.com/blog/pandas-pyarrow-parquet-s3/#reading-parquet-file-from-s3-as-pandas-dataframe

In [ ]:
# convert to geojson and return